In [14]:
import sys
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
sys.path.append("..")

# news scraping
from src.zinfo.news_scraper import NewsScraper

# clustering
from src.zinfo.article_clustering import get_vectorized_titles
from src.zinfo.article_clustering import get_num_clusters_per_val
from src.zinfo.article_clustering import cluster_articles

# article selecting
from src.zinfo.article_selector import get_best_article_all_clusters

# when I officially started running the algorithm
start_date = "2021-08-13"

In [16]:
all_news = pd.read_csv("../data/all_news.csv")
selected_news = pd.read_csv("../data/selected_articles.csv")
sources = selected_news.source.value_counts().to_dict()

In [18]:
all_news

,date,title,url,source,topic
0,2021-08-12T20:00:24Z,Open Post: Hosted By Brendan Fraser Getting Em...,https://dlisted.com/2021/08/12/open-post-hoste...,NaN,Brendan Fraser
1,2021-08-12T14:45:00Z,"Sometimes, the Internet is Good: Brendan Frase...",https://www.pajiba.com/web_culture/sometimes-t...,NaN,Brendan Fraser
2,2021-08-12T17:30:38Z,10 Brilliant Martin Scorsese Movies to Buy on ...,https://www.indiewire.com/shop/best-martin-sco...,NaN,Brendan Fraser
3,2021-08-12T20:29:00Z,Watch: Brendan Fraser Gets Emotional in Respon...,https://comicbook.com/movies/news/brendan-fras...,NaN,Brendan Fraser
4,2021-08-12T21:05:00Z,GreenPower Reports Fiscal First Quarter 2022 F...,https://finance.yahoo.com/news/greenpower-repo...,NaN,Brendan Fraser
...,...,...,...,...,...
12006,2021-08-26T07:11:34Z,AP News Digest 3:10 am,https://www.independent.co.uk/news/world/ameri...,Independent,Nirvana Nevermind
12007,2021-08-26T14:20:33Z,Porn star Ron Jeremy indicted on more than 30 ...,https://globalnews.ca/news/8142376/ron-jeremy-...,Global News,Nirvana Nevermind
12008,2021-08-26T09:13:16Z,Glory Days! Remembering the finest gigs Cork's...,https://www.echolive.ie/entertainment/whatson/...,Echo Live,Nirvana Nevermind
12009,2021-08-26T11:10:40Z,"Baby from Nirvana album cover now suing band, ...",https://abc7ny.com/baby-sues-nirvana-nevermind...,WABC-TV,Nirvana Nevermind


In [19]:
selected_news

,date,title,url,source,topic,num_articles
0,2021-08-11,"'Hard Knocks' Power Rankings: Dak Prescott, Mi...",https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,Hard Knocks 2021,47
1,2021-08-11,NFL playoff teams that could miss postseason i...,https://www.cbssports.com/nfl/news/nfl-playoff...,NaN,Hard Knocks 2021,4
2,2021-08-11,Chris Cuomo Defied CNN Ban On Advising His Sex...,http://www.womensystems.com/2021/08/chris-cuom...,NaN,Kerry Kennedy,271
3,2021-08-11,Messi joins PSG: Lionel Messi talks about play...,https://www.insidesport.co/messi-joins-psg-lio...,NaN,PSG,83
4,2021-08-11,Christina Applegate announces multiple scleros...,https://www.scotsman.com/health/christina-appl...,NaN,Christina Applegate,4
...,...,...,...,...,...,...
268,2021-08-26,Monica Lewinsky insisted thong-flashing scene ...,https://www.dailymail.co.uk/femail/article-993...,Daily Mail,Monica Lewinsky,8
269,2021-08-26,Tropical Storm Ida forecast to pose major hurr...,https://www.axios.com/tropical-storm-ida-hurri...,Axios,Hurricane,9
270,2021-08-26,Man photographed as baby for Nirvana 'Nevermin...,https://www.wfaa.com/article/news/nation-world...,WFAA.com,Nirvana Nevermind,14
271,2021-08-26,Manchester City's Benjamin Mendy charged with ...,https://www.cbssports.com/soccer/news/manchest...,CBS Sports,Benjamin Mendy,35


In [20]:
sources

{'Yahoo Entertainment': 17,
 'Independent': 11,
 'USA Today': 10,
 'CBS Sports': 5,
 'Slashdot.org': 5,
 'The Daily Caller': 4,
 'Bleacher Report': 4,
 'CBC News': 3,
 'Business Insider': 3,
 'Fox News': 3,
 'Variety': 3,
 'ComicBook.com': 3,
 'Bloomberg': 3,
 'HYPEBEAST': 3,
 'TheWrap': 3,
 'CBS News': 3,
 'Stereogum': 3,
 'The Guardian': 3,
 'Daily Mail': 3,
 'NPR': 3,
 'Reuters': 2,
 'Thatsnerdalicious.com': 2,
 'The Boston Globe': 2,
 'Worldsoccertalk.com': 2,
 'HuffPost': 2,
 'GameSpot': 2,
 'EURACTIV': 2,
 'MMA Fighting': 2,
 'Newsweek': 2,
 'NBC News': 2,
 'Access': 2,
 'NBCSports.com': 2,
 'Independent.ie': 2,
 'The Hill': 2,
 'New York Post': 2,
 'ETCanada.com': 2,
 'CNA': 2,
 'U.S. Soccer': 2,
 'Billings Gazette': 2,
 'Eonline.com': 2,
 'The Week Magazine': 2,
 'Destructoid': 2,
 'Page Six': 2,
 'Associated Press': 2,
 'The Times of India': 2,
 'Al Jazeera English': 2,
 'Www.https': 2,
 'BBC News': 1,
 'The Weather Channel': 1,
 'The Washington Post': 1,
 '4029tv': 1,
 'The-s

## Tracking trending keywords over multiple days

In [63]:
remove_dup_news = selected_news.drop_duplicates(subset=["date", "topic"])
topics = remove_dup_news.topic.value_counts()
topics = topics.loc[topics >= 2].index.to_list()
long_trends = remove_dup_news.loc[remove_dup_news.topic.isin(topics)].copy()
long_trends.sort_values(by="topic", inplace=True)

In [67]:
long_trends.set_index(["topic", "date"])

title  \
topic                  date                                                            
49ers                  2021-08-23  Kyle Shanahan still won’t name 49ers’ starting...   
                       2021-08-15  Former Seahawks linebacker Mychal Kendricks to...   
Barcelona              2021-08-16  Soccer-Barca can escape financial hole in 18 m...   
                       2021-08-21         Depay rescues draw for Barcelona in Bilbao   
Bob Dylan              2021-08-17   Bob Dylan sued for alleged sexual abuse of minor   
                       2021-08-16  Bob Dylan sued for allegedly sexually abusing ...   
CM Punk                2021-08-26  CM Punk Teases Daniel Bryan During His AEW Dyn...   
                       2021-08-21  CM Punk Sells Free Ice Cream Bars to Fans Atte...   
COVID booster shot     2021-08-18          What We Know About COVID-19 Booster Shots   
                       2021-08-17  US experts expected to recommend COVID-19 vacc...   
Call of Duty: Vanguard 2021-08-19  Call of Duty: Warzone will finally get a new a...   
                       2021-08-16  Call Of Duty: Vanguard Official Reveal Droppin...   
Chelsea                2021-08-12  Romelu Lukaku Rejoins Chelsea For Club Record ...   
                       2021-08-22  Soccer-Returning hero Lukaku sets Chelsea on w...   
Cowboy Bebop           2021-08-24  Netflix's live-action 'Cowboy Bebop' series ar...   
                       2021-08-23  Netflix’s ‘Cowboy Bebop’ Live-Action Series Se...   
Don Everly             2021-08-23  Don Everly of early rock ‘n’ roll Everly Broth...   
                       2021-08-22             Everly Brothers’ Don Everly Dead At 84   
Eric Stonestreet       2021-08-26  Modern Family star Eric Stonestreet hits back ...   
                       2021-08-23  Eric Stonestreet is engaged to fiancée Lindsay...   
Eternals               2021-08-19            Epic New Trailer For Marvel's ETERNALS!   
                       2021-08-20  Angelina Jolie joins Instagram to share letter...   
Greg Abbott            2021-08-17  Texas Gov. Greg Abbott, who banned mask mandat...   
                       2021-08-18  Texas Governor Greg Abbott tests positive for ...   
Havana Syndrome        2021-08-24  Kamala Harris' flight to Vietnam delayed for t...   
                       2021-08-25  China state media says U.S. VP Harris seeking ...   
Jimmy Hayes            2021-08-23           Former NHL player Jimmy Hayes dies at 31   
                       2021-08-24  Tributes paid to former NHL player Jimmy Hayes...   
Mike Richards          2021-08-21  Mike Richards Steps Down As New Host of ‘Jeopa...   
                       2021-08-20   Mike Richards steps down as new 'Jeopardy!' host   
OnlyFans               2021-08-25  OnlyFans Drops Plan to Ban Sexually Explicit C...   
                       2021-08-19  OnlyFans Will Ban All Sexually Explicit Conten...   
                       2021-08-20  OnlyFans will ban "sexually explicit'" content...   
PSG                    2021-08-14  PSG fans give Messi huge ovation; Galtier's Ni...   
                       2021-08-15  Barcelona start post-Messi era with win over R...   
                       2021-08-11  Messi joins PSG: Lionel Messi talks about play...   
Packers                2021-08-14  Aaron Rodgers, others plead with Packers to si...   
                       2021-08-15      'Top 100 Players of 2021': Jason Kelce No. 92   
Pfizer vaccine         2021-08-24  Why Pfizer's FDA Approval Matters And What It ...   
                       2021-08-23  FDA grants full approval to Pfizer-BioNTech Co...   
Premier League         2021-08-13  Virgil van Dijk: Liverpool defender signs deal...   
                       2021-08-14  Soccer-Watford hold on to make winning Premier...   
Real Madrid            2021-08-14  Manchester United complete signing of Raphael ...   
                       2021-08-23  Argentina call up Messi for World Cup qualifie...   
Reminiscence           2021-08-